<a href="https://colab.research.google.com/github/Fabiorsi1979/FTC_Analisando_dados/blob/main/Visao_Restaurante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install haversine

In [4]:
from google.colab import drive
import pandas as pd
from haversine import haversine


drive.mount('/content/drive')

path = '/content/drive/MyDrive/train_limpo.csv'



Mounted at /content/drive


In [6]:
dados = pd.read_csv(path)
dados.head(3)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26


In [7]:
df= dados.copy()

# Visão Restaurante

### A quantidade de entregadores únicos.

In [12]:
print('Temos {} entregadores únicos no cadastro'.format(len(df.Delivery_person_ID.unique())))

Temos 1320 entregadores únicos no cadastro


### A distância média dos resturantes e dos locais de entrega.


In [19]:
cols = ['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']

df['Distance'] = df.loc[0:10,cols].apply(lambda x: haversine ((x['Restaurant_latitude'],x['Restaurant_longitude']),(x['Delivery_location_latitude'],x['Delivery_location_longitude'])),axis=1)

df.Distance.mean()


10.606957967616198

### O tempo médio e o desvio padrão de entrega por cidade.


In [23]:
df_aux = df.loc[:,['Time_taken(min)','City']].groupby('City').agg(['mean','std']).reset_index()

df_aux.columns = ['Cidade','Tempo Médio minutos','Desvio Padrão']

df_aux

,Cidade,Tempo Médio minutos,Desvio Padrão
0,Metropolitian,27.136486,9.205358
1,Semi-Urban,49.731707,2.693089
2,Urban,22.984017,8.866011


### O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.

In [25]:
df.loc[:,['City','Type_of_order','Time_taken(min)']].groupby(['City','Type_of_order']).agg({'Time_taken(min)':['mean','std']}).reset_index()

City Type_of_order Time_taken(min)          
                                           mean       std
0   Metropolitian        Buffet       27.013173  9.224802
1   Metropolitian        Drinks       27.037381  9.103204
2   Metropolitian          Meal       27.334937  9.272359
3   Metropolitian         Snack       27.155229  9.216312
4      Semi-Urban        Buffet       49.666667  2.711013
5      Semi-Urban        Drinks       49.647059  2.435624
6      Semi-Urban          Meal       50.029412  2.959170
7      Semi-Urban         Snack       49.648148  2.720692
8           Urban        Buffet       23.352073  9.081076
9           Urban        Drinks       22.991430  8.933587
10          Urban          Meal       22.802414  8.745737
11          Urban         Snack       22.793478  8.693936

### O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.

In [27]:
df.loc[:,['City','Time_taken(min)','Road_traffic_density']].groupby(['City','Road_traffic_density']).agg({'Time_taken(min)':['mean','std']}).reset_index()

City Road_traffic_density Time_taken(min)           
                                                  mean        std
0   Metropolitian                 High       27.977765   8.023710
1   Metropolitian                  Jam       31.747122   9.588640
2   Metropolitian                  Low       22.215688   7.019964
3   Metropolitian               Medium       27.488228   8.388722
4      Semi-Urban                 High       50.058824   2.561020
5      Semi-Urban                  Jam       49.888889   2.697889
6      Semi-Urban                  Low       48.000000        NaN
7      Semi-Urban               Medium       47.454545   1.916436
8           Urban                 High       24.125926   8.547723
9           Urban                  Jam       27.743434  10.108344
10          Urban                  Low       19.338165   6.429146
11          Urban               Medium       23.730900   8.433615

### O tempo médio de entrega durantes os Festivais.


In [30]:
df_aux = df[df['Festival']=='Yes']

df_aux['Time_taken(min)'].mean()

45.517857142857146